In [34]:
import cv2 as cv  #OpenCV Kütüphanesi
import os
import random
import numpy as np
import pickle

Not: Pickle : Elimdeki dosyaları bilgisayarıma kaydetmeme yarayan bir kütüphane.

In [19]:
xPath = "../Data_Project/Spectrograms"
images = []

number_to_error = 0
number_to_processed = 0

errors_file = open("images_errors.txt","w")
limit = 0
for classid in os.listdir(xPath):
    #print(classid)
    path_to_class = f"{xPath}/{classid}"
    for image in os.listdir(path_to_class):
        if limit >= 2500:
            break
        try:
            path_to_image = f"{path_to_class}/{image}"
            #print(f"{path_to_image}")
            img = cv.imread(path_to_image, 0)
            img = cv.resize(img, (128,128))  #resizeing yapıyoruz.
            img = img/255  #normalizasyon yapıyoruz.
            images.append([img, int(classid)])
        except Exception as e:
            number_to_error += 1
            errors_file.write(f"{number_to_processed} --- {e}")
        finally:
            number_to_processed += 1

        print(f"\rProcessed : {number_to_processed} / Error : {number_to_error}", end="")
        limit += 1
errors_file.close()

Processed : 2500 / Error : 0

Not 1 : Neden resize yapıyoruz? : Çünkü boyutu küçülterek kolay ve hızlı işlem yapma imkanımızı artırıyouz. Elbette resimde bazı kaıplar oluşuyor ancak bunlar kabul edilebilir kayıplar. Resmin özniteliğini bozmadan oluşan kayıplara karşı daha başka imkanlar elde edebiliyoruz. Bu da projeden projeye değişebilen bir faktör. 

Not 2 : Neden spectrogramları siyah beyaz a çeviriyoruz? Örnek çalışmada yer alan değişkenlerden : img.shape --> (374, 500, 3)  (renkli)  --> 374 x 500 x 3 = 561.000 okunacak pixel sayısı | img_gray.shape -->(374,500)  (siyah-Beyaz) --> 374 x 500 = 187.000 okunacak pixel sayısı | Siyah beyaz specktrogramlarda aynı öznitelikteki resimleri her hangi bir kayba uğramadan 3 kat az pixel sayısına göre işlem yapılabildiği iiçin tercih ediyoruz.

Not 3 : Normalizasyon Nedir? - Bir veri setinin 0 ile 1 arasına tüm değerlerinin getirilmiş olmasıdır.

#### Örnek Çalışma 

In [20]:
#img = cv.imread("../Data_Project/Spectrograms/0/100852-0-0-0.png",1)
#img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

#img.shape

#cv.imshow("frame",img)
#cv.imshow("frame_gray",img_gray)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [21]:
len(images)

2500

Not : images değerini 10 kere karıştırıyoruz.

In [22]:
for _ in range(10):
    random.shuffle(images)

In [23]:
images[0][1]

0

In [24]:
X=[]
y=[]

for image, classid in images:
    X.append(image)
    y.append(classid)

In [25]:
print(len(X))
print(len(y))

2500
2500


Not: Elimizdeki veriyi train, validation ve test verisi olarak bölelim.

In [26]:
X_train = X[:2000]
y_train = y[:2000]

X_val = X[2000:2250]
y_val = y[2000:2250]

X_test = X[2250:]
y_test = y[2250:]

In [29]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

2000
250
250


Not: Bulduğumuz değerleri Numpy dizisine çevirelim.

In [27]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)

X_test = np.array(X_test)
y_test = np.array(y_test)

Sadece tek bir görüntüyü yazdırırsak

In [30]:
print(X_train[0].shape)

(128, 128)


Görüntülerin tamamını yazdırırsak : 2000 görsel var ve boyutlar 128x128

In [31]:
print(X_train.shape)

(2000, 128, 128)


Not: TensorFlow da tf.keras.layers.Conv2D altında bizden istenilen argümanlar içerisinde kanal sayısı da var. Bu nedenle elimizdeki dataya kanal sayısı ekliyoruz.
reshape metodunda -1 konulduğunda sample sayısı bilinmiyoruz demektir. reshape bunu kendisi belirler. Burada aslında 2000 yazabilirdik. Ama bilmediğimizi farzediyoruz.
Bizden "sample sayısı", "genişlikler", "yükseklikler" ve "kanal sayısı" olarak belirtmemiz isteniyor.

In [32]:
X_train = X_train.reshape(-1,128,128,1)
X_val = X_val.reshape(-1,128,128,1)
X_test = X_test.reshape(-1,128,128,1)

In [33]:
print(X_train.shape)

(2000, 128, 128, 1)


In [36]:
if "pickles" not in os.listdir():
    os.mkdir("pickles")

with open("pickles/X_train.pickle","wb") as f:
    pickle.dump(X_train, f)
with open("pickles/y_train.pickle","wb") as f:
    pickle.dump(y_train, f)

with open("pickles/X_val.pickle","wb") as f:
    pickle.dump(X_val, f)
with open("pickles/y_val.pickle","wb") as f:
    pickle.dump(y_val, f)

with open("pickles/X_test.pickle","wb") as f:
    pickle.dump(X_test, f)
with open("pickles/y_test.pickle","wb") as f:
    pickle.dump(y_test, f)
    